# Example Ground Irradiance with PySAM and bifacialVF

## Option 1, pysam

In [1]:
import PySAM
import PySAM.Pvsamv1 as PV
import PySAM.Grid as Grid
import PySAM.Utilityrate5 as UtilityRate
import PySAM.Cashloan as Cashloan
import pathlib, os
import json
import pvlib
import pandas as pd

In [2]:
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("Pvlib version: ", pvlib.__version__)
print("PySAM version: ", PySAM.__version__)

Working on a  Windows 11
Python version  3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Pandas version  2.2.2
Pvlib version:  0.10.3
PySAM version:  6.0.0


In [3]:
sif4 = 'Row4Json'

jsonnames = ['Row4LongiBifi']

file_names = ["pvsamv1", "grid", "utilityrate5", "cashloan"]

pv4 = PV.new()  # also tried PVWattsSingleOwner
grid4 = Grid.from_existing(pv4)
ur4 = UtilityRate.from_existing(pv4)
so4 = Cashloan.from_existing(grid4, 'FlatPlatePVCommercial')


In [4]:
for count, module in enumerate([pv4, grid4, ur4, so4]):
    filetitle= 'Row4LongiBifi' + '_' + file_names[count] + ".json"
    with open(os.path.join(sif4,filetitle), 'r') as file:
        data = json.load(file)
        for k, v in data.items():
            if k == 'number_inputs':
                continue
            try:
                if sys.version.split(' ')[0] == '3.11.7': 
                    # Check needed for python 3.10.7 and perhaps other releases above 3.10.4.
                    # This prevents the failure "UnicodeDecodeError: 'utf-8' codec can't decode byte... 
                    # This bug will be fixed on a newer version of pysam (currently not working on 5.1.0)
                    if 'adjust_' in k:  # This check is needed for Python 3.10.7 and some others. Not needed for 3.7.4
                        print(k)
                        k = k.split('adjust_')[1]
                module.value(k, v)
            except AttributeError:
                # there is an error is setting the value for ppa_escalation
                print(module, k, v)

<Pvsamv1 object at 0x000001AA7B9C5FE0> adjust_constant 0
<Pvsamv1 object at 0x000001AA7B9C5FE0> adjust_en_timeindex 0
<Pvsamv1 object at 0x000001AA7B9C5FE0> adjust_en_periods 0
<Pvsamv1 object at 0x000001AA7B9C5FE0> adjust_timeindex [0]
<Pvsamv1 object at 0x000001AA7B9C5FE0> adjust_periods [[0, 0, 0]]
<Pvsamv1 object at 0x000001AA7B9C5FE0> dc_adjust_constant 0
<Pvsamv1 object at 0x000001AA7B9C5FE0> dc_adjust_en_timeindex 0
<Pvsamv1 object at 0x000001AA7B9C5FE0> dc_adjust_en_periods 0
<Pvsamv1 object at 0x000001AA7B9C5FE0> dc_adjust_timeindex [0]
<Pvsamv1 object at 0x000001AA7B9C5FE0> dc_adjust_periods [[0, 0, 0]]


##### Sanity checks

In [5]:
# For unknown reasons, pySAM does not calculate this number and you have to obtain it from the GUI.
system_capacity4 = 73.982   

In [ ]:
# Change any variable you might want here. 
# Example:
#pv4.SolarResource.solar_resource_file = weatherfile

# Other ones I change often
#pv4.SolarResource.use_wf_albedo
#pv4.SolarResource.irrad_mode
#pv4.SolarResource.albedo
#pv4.SolarResource.irrad_mode = 3 # [0/1/2/3/4] Beam+Diff,Global+Beam, Global+Diff, POA Ref cell, POA Pyranometer
#pv4.SolarResource.sky_model= 2 #  [0/1/2] Isotropic,HDKR,Perez

# If using POA sensors as input, shading has to be inactivated.
#if pv4.SolarResource.irrad_mode >= 3:
#    pv4.Shading.subarray1_shade_mode = 0
#else:
#    pv4.Shading.subarray1_shade_mode = 1.0

# Using custom tracker angles
# pv4.SystemDesign.subarray1_use_custom_rot_angles = 1
# pv4.SystemDesign.subarray1_custom_rot_angles_array = trackerangles # MUST BE same length as weatherfile.

In [7]:
# Execute calculations
grid4.SystemOutput.gen = [0] * 8760  # p_out   # let's set all the values to 0
pv4.execute()
grid4.execute()
ur4.execute()
so4.execute()

In [24]:
# SAVE RESULTS|
results = pv4.Outputs.export()
power4 = list(results['subarray1_dc_gross']) # normalizing by the system_capacity
celltemp4 = list(results['subarray1_celltemp'])
rear4 = list(results['subarray1_poa_rear'])
front4 = list(results['subarray1_poa_front'])
custom_angles = list(results["subarray1_axisrot"])

dni = list(results['dn'])
dhi = list(results['df'])
alb = list(results['alb'])

res = pd.DataFrame(list(zip(power4, celltemp4, rear4, front4,
                          dni, dhi, alb)),
       columns = ['Power4' , 'CellTemp4', 'Rear4', 'Front4',
                 'DNI', 'DHI', 'Alb'])
#res['index'] = res.index
res['Power4']= res['Power4']/system_capacity4 # normalizing by the system_capacity
#    res.index = timestamps
res = res.iloc[0:8760,:]


In [73]:
ground_irradiance = results["subarray1_ground_rear_spatial"]

# redo this using numba?
distances = ground_irradiance[0][1:]
ground_irradiance_values = pd.DataFrame(ground_irradiance[1:][:])
ground_irradiance_values.drop(df.columns[0], axis=1, inplace=True)

In [80]:
joined_df = res.join(ground_irradiance_values, how='inner')
times = pd.date_range(start="2025-01-01", periods=8760, freq="1h")
joined_df.index = times
joined_df

,Power4,CellTemp4,Rear4,Front4,DNI,DHI,Alb,1,2,3,4,5,6,7,8,9,10
2025-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-12-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
joined_df.to_pickle('Results.pkl')
joined_df.to_csv('Results.csv', float_format='%g')


## OPTION 2

Only works for fixed tilt, but gives 100 ground irradiance values so higher resolution

In [84]:
!pip install bifacialvf

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
     ----- ---------------------------------- 1.3/9.5 MB 8.4 MB/s eta 0:00:01
     --------------- ------------------------ 3.7/9.5 MB 9.9 MB/s eta 0:00:01
     --------------------------- ------------ 6.6/9.5 MB 11.2 MB/s eta 0:00:01
     ---------------------------------------  9.4/9.5 MB 12.2 MB/s eta 0:00:01
     ---------------------------------------- 9.5/9.5 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bifacialvf: filename=bifacialvf-0.1.9-py2.py3-none-any.whl size=7257130 sha256=9b9b37d4cfe3caab20de848913d904c6d551dde123df6ea041c4d7262f625aff
  Stored in directory: c:\users\sayala\appdata\local\pip\cache\wheels\7e\42\e0\854dc730bf6a6870d241067f98cf982c5fad730253455aa5d4
Successfully built bifacialvf


In [85]:
import bifacialvf


In [87]:
print("bifacialvf version ", bifacialvf.__version__)

bifacialvf version  0.1.9


In [90]:
writefiletitle = os.path.join('Results_bifacialVF.csv')

# Variables
lat = 39.75555              # Golden CO, Coords.
lon = -105.2211             # Golden CO, Coords.
tilt = 10                   # PV tilt (deg)
sazm = 180                  # PV Azimuth(deg) or tracker axis direction
albedo = None               # Calculated in previous section from SRRL data. Value is 0.28 up to 11/18/19o
hub_height=1.5/2            #1.5m / 2m collector width
pitch = 2/0.35/2              # 1 / 0.35 where 0.35 is gcr --- row to row spacing in normalized panel lengths. 
rowType = "interior"        # RowType(first interior last single)
transFactor = 0             # TransmissionFactor(open area fraction)
sensorsy = 12                # sensorsy(# hor rows in panel)   <--> THIS ASSUMES LANDSCAPE ORIENTATION 
PVfrontSurface = "glass"    # PVfrontSurface(glass or ARglass)
PVbackSurface = "glass"     # PVbackSurface(glass or ARglass)
agriPV = True

# Tracking instructions
tracking=False   # Ground irradiance for tracking needs improvement; fixed works well.
backtrack=False
limit_angle = 60

# Download and Read input
TMYtoread=bifacialvf.getEPW(lat=lat,lon=lon)
myTMY3, meta = bifacialvf.readInputTMY(TMYtoread)
deltastyle = 'TMY3'
myTMY3 = myTMY3.iloc[0:24].copy()  # Simulate just the first 24 hours of the data file for speed on this example

bifacialvf.simulate(myTMY3, meta, writefiletitle=writefiletitle, 
         tilt=tilt, sazm=sazm, pitch=pitch, hub_height=hub_height, 
         rowType=rowType, transFactor=transFactor, sensorsy=sensorsy, 
         PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, 
         albedo=albedo, tracking=tracking, backtrack=backtrack, 
         limit_angle=limit_angle, deltastyle=deltastyle, agriPV=agriPV)



path = C:\Users\sayala\Documents\GitHub\Tutorials\pySAMtutorial\4 - Passing POA through pySAM
Making path: EPWs
Getting weather file: USA_CO_Golden-NREL.724666_TMY3.epw
 ... OK!
Calculating Sun position with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Using albedo from TMY3 file.
Note that at the moment, no validation check is done in the albedo data, so we assume it's correct and valid.

 
********* 
Running Simulation for TMY3: 
Location:   Denver Centennial  Golden   Nr
Lat:  39.74  Long:  -105.18  Tz  -7.0
Parameters: tilt:  10   Sazm:  180     Clearance_Height :  0.75   Pitch:  2.857142857142857   Row type:  interior   Albedo:  None
Saving into Results_Tutorial2.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.4841993245570035
Actual distance between rows =  1.872335104130649
 
Saving Ground Irradiance Values for AgriPV Analysis. 


  0%|                                                                                                                               | 0/24 [00:00<?, ?it/s]C:\Users\sayala\AppData\Roaming\Python\Python312\site-packages\bifacialvf\bifacialvf.py:459: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  albedo = myTMY3.Alb[rl]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 113.75it/s]

Finished


In [91]:
#Load the results from the resultfile
from bifacialvf import loadVFresults
(data, metadata) = loadVFresults(writefiletitle)

In [96]:
data['Ground Irradiance Values']

0    [1.1401204692731868 1.1147534024273793 1.09002...
1    [42.64438828134228 41.69557359264453 40.770546...
2    [27.932951497193077 27.311458359470794 26.7055...
3    [64.28436555296548 62.85407301628121 378.70431...
4    [268.27050554616375 266.55933255460263 264.891...
5    [182.25051998966111 180.08788646062425 177.979...
6    [81.08945319464584 96.94972353160227 95.190758...
7    [33.63355384355901 32.88522537160769 32.155657...
8    [28.56360708163836 27.928082196571427 27.30849...
Name: Ground Irradiance Values, dtype: object